# Llamaindex connector

In [1]:
import os
import logging
import sys
from openai import OpenAI
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from llama_index import (
    VectorStoreIndex, 
    SimpleDirectoryReader, 
    LLMPredictor, 
    ServiceContext, 
    StorageContext, 
    load_index_from_storage
    )


# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Load the environment variables
load_dotenv()

OpenAI.api_key = os.getenv("OPENAI_API_KEY")

Generate the embeddings and store them

In [2]:
# check if storage already exists
if not os.path.exists("./storage"):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    # Save embeddings to disk 
    index.storage_context.persist()
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    index = load_index_from_storage(storage_context)

Integrate the LLM

In [5]:
llm_predictor = LLMPredictor(
    llm = ChatOpenAI(
        temperature=0.2,
        model_name ="gpt-3.5-turbo",
        openai_api_key= os.getenv("OPENAI_API_KEY")
        )
    )

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

query_engine = index.as_query_engine(service_context=service_context)

question = "What did the author do growing up"

response = query_engine.query(question)

print(response)

The author mentions that before college, they worked on writing and programming. They wrote short stories and also tried writing programs on the IBM 1401 computer. They also mention getting a microcomputer, a TRS-80, and started programming on it.
